# Care Homes

In [2]:
import polars as pl
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import pickle
import io
import boto3
import re
import os
from enum import Enum
from sklearn.base import BaseEstimator

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
import sys
notebook_dir = os.getcwd()

project_root = os.path.dirname(notebook_dir)

sys.path.insert(0, project_root)

from utilities.version import ModelVersionManager

## Define S3 bucket and Location of the Data

In [6]:
BUCKET = 'sfc-mt-sagemaker-demo'                                                                                    #used for demo purposes 
CAREHOMEDATA = 's3://sfc-main-datasets/domain=ind_cqc_filled_posts/dataset=ind_cqc_estimated_missing_ascwds_filled_posts/'

## Read and Filter Dataset

In [7]:
df1 = pl.scan_parquet(CAREHOMEDATA)

In [10]:
locations_df = df1.collect()

In [17]:
def calculate_care_home_status_count(df):
    w = df.partition_by("locationId")

    df = df.with_columns(
        "care_home_status_count",
        F.size((F.collect_set("carehome").over(w))),
    )
    return df

    .filter(pl.col("careHome") == "N")

In [18]:
locations_df = calculate_care_home_status_count(locations_df)

AttributeError: 'DataFrame' object has no attribute 'withColumn'

In [ ]:
filtered_loc_data = locations_df.filter(
    (F.col("carehome") == 'Y') & (F.col("care_home_status_count") == 1))